In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q accelerate
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [ ]:
from datasets import load_dataset
meetingbank = load_dataset("huuuyeah/meetingbank")

In [ ]:
train_data = meetingbank['train']
print(train_data)

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.summarize import load_summarize_chain

In [ ]:
dataset_name = "huuuyeah/meetingbank"
page_content_column = "transcript"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

In [ ]:
data = loader.load()
data[:10]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()
docs = splitter.split_documents(data[:1])

In [ ]:
import torch

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    # quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )


In [ ]:
from langchain.chains.summarize import load_summarize_chain

prompt_template = """[INST]
Act as a professional technical meeting minutes writer.
Tone: formal
Format: Technical meeting summary
Use bullet points to express key points


{text}


CONCISE SUMMARY IN ITALIAN:
[/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

print(chain.run(docs[12:13]))